In [1]:
from make_dataset import make_dataset
from artificial_neural_network import ArtificialNeuralNetwork
#from make_train_test_dataset import MakeTrainTestDataset
#from reshape_merger_tree import ReshapeMergerTree
import numpy as np
import copy as cp
import os, sys
import pickle

In [2]:
##Print a dir_list which is a list of ectracted parameters.
##The parameters are made by ./parameter/MakePickleOfParam.ipynb.
dir_list = np.loadtxt("param_list.txt", dtype = "str")
for i in range(len(dir_list)):
    if i == 0:
        print(dir_list[i])
    else:
        print("{}. {}".format(i, dir_list[i]))

--Parameters--
1. param_1e+7_1e+18_ScaleFactor_ID_pid_Mvir_Rvir_x_vx_MW039_MW038_MW035_MW034_MW033_MW032.pickle


In [3]:
##Input index-number of param_list.txt in dir_num to select it.
##Load the parameters of host-halo and sub-halo.
dir_num = int(input("\nSelect index>> "))
with open("parameters/" + dir_list[dir_num], mode = "rb") as f:
    param = pickle.load(f)
with open("parameters/host_" + dir_list[dir_num], mode = "rb") as f:
    host_param = pickle.load(f)


Select index>> 1


In [4]:
##The list of used parameters and host-halo.
param_list = list(param.keys())
mainbranch_list = list(param[param_list[0]].keys())
print("parameters : {}".format(param_list))
print("files : {}".format(mainbranch_list))

parameters : ['ScaleFactor', 'ID', 'pid', 'Mvir', 'Rvir', 'x', 'vx']
files : ['mainbranch_MW039.csv', 'mainbranch_MW038.csv', 'mainbranch_MW035.csv', 'mainbranch_MW034.csv', 'mainbranch_MW033.csv', 'mainbranch_MW032.csv']


In [5]:
##Get zacc of each sub-halo as acc_sf(Data type is dictionary).
##The zacc is a time when ID of host-halo and pid of sub-halo match.
acc_sf = {}
for m_key in mainbranch_list:
    acc_sf[m_key] = []
    for idx, parameter in enumerate(param["pid"][m_key]):
        for i in range(parameter.size):
            host_i = host_param["ID"][m_key].size - parameter.size + i
            if host_param["ID"][m_key][host_i] == parameter[i]:
                acc_sf[m_key].append(i)
                break
            elif i == parameter.size-1:
                acc_sf[m_key].append(-1)

In [6]:
##Describe variables of learning-parameter of a DeepLearningModel in LearnParam.
class LearnParam:
    def __init__(self):
        self.eps = 1e-7  ##Prevent devision by zero like 1 / (0 + eps).
        self.bias = False ##If This bias is True, add normalized dataset to bias.
        self.hidden = [100]*10  ##Defines number of hidden layers and neurons in each layers.
        self.batchsize_denominator = 100  ##Defines a size of mini-batch as size(train-dataset) /  batch_denominator.
        self.learning_rate = "1e-3"  ##Learning rate of Back propagation.
        self.optimizer = "Adam"  ##Optimizer of the DeepLearningModel.
        self.batch_normalization = True  ##Whether to use BatchNormalization or not.
        self.loss_func = "MSE_RE"  ##Loss function of a DeepLearning model.
        self.activation_func = "tanhexp"  ##Activation function of the hidden layers.
        self.weight_init = "he"  ##Condition for initializing weight of the hidden layers.
        self.lastlayer_identity = True
        self.epoch = 200  ##Training epoch.
        self.input_size = 3  ##Size of input-dataset(axis == 1)
        self.output_size = 7  ##Size of output-dataset(axis == 1)
        self.param_kind = "x"  ##Learning parameter.
        self.train_ratio = 0.9  ##Percentage of training-dataset to all-dataset.
        self.fft_format = "rfft"
        self.normalize_format = "Standardization"  ##Selects a format of dataset normalization in None, Normalization, Standardization.
        self.extract_dataset = "After_acc"  ##Selects a condition of dataset.
        ##All is overall, Before/After_acc is in part of before/after accretion, All(_not)_acc is overall of (not)accreted sub-halo.
        self.learn_num = 1  ##Distinguishes each directories where training results are saved.
        self.save_fig_type = ".png"  ##Format of the saved figures.

In [7]:
LP = LearnParam()

In [8]:
dataset = make_dataset(mainbranch_list, LP.param_kind, host_param, param, LP.extract_dataset, acc_sf, LP.input_size, LP.output_size)
#MTTD = MakeTrainTestDataset(mainbranch_list)
#train, test = MTTD.split(cp.deepcopy(dataset), LP.train_ratio)

In [9]:
ANN = ArtificialNeuralNetwork(LP.input_size, LP.hidden, LP.activation_func, LP.weight_init, LP.batch_normalization, LP.output_size, LP.lastlayer_identity, LP.loss_func, is_epoch_in_each_mlist = True)

In [10]:
ANN.learning(dataset, LP.train_ratio, LP.optimizer, LP.learning_rate, LP.batchsize_denominator, LP.epoch, LP.fft_format, LP.normalize_format)

Make a train/test dataset.
Mini-batch size : 63
Iterations per 1epoch : 100
Total iterations : 20000


KeyboardInterrupt: 

In [9]:
data_input, data_output = None, None
RMT = {}
for m_key in mainbranch_list:
    RMT[m_key] = ReshapeMergerTree()
    input_, output_ = RMT[m_key].make_dataset(dataset[m_key], LP.input_size, LP.output_size)
    if data_input is None and data_output is None:
        data_input = input_
        data_output = output_
    else:
        data_input = np.concatenate([data_input, input_], axis = 0)
        data_output = np.concatenate([data_output, output_], axis = 0)

In [42]:
data_unit = np.concatenate([data_input[:, :LP.input_size], data_output, data_input[:, LP.input_size:]], axis = 1)

In [43]:
#fft_input = np.fft.fft(data_input)
#ifft_input = np.fft.ifft(fft_input)
fft_input = np.fft.rfft(data_input)
ifft_input = np.fft.irfft(fft_input)

In [44]:
print(ifft_input[:3] - data_input[:3])

[[ 2.96984659e-15 -1.17961196e-15  0.00000000e+00  3.55271368e-15
   3.55271368e-15  7.10542736e-15]
 [ 5.96744876e-16 -4.13558077e-15  0.00000000e+00  3.55271368e-15
   3.55271368e-15  3.55271368e-15]
 [-1.77635684e-15  0.00000000e+00 -5.32907052e-15  0.00000000e+00
   0.00000000e+00  3.55271368e-15]]


In [45]:
#fft_unit = np.fft.fft(data_unit)
#ifft_unit = np.fft.ifft(fft_unit)
fft_unit = np.fft.rfft(data_unit)
ifft_unit = np.fft.irfft(fft_unit)

In [47]:
print(ifft_unit[:3] - data_unit[:3])

[[ 7.63278329e-16  3.55271368e-15  1.77635684e-15  0.00000000e+00
   0.00000000e+00  0.00000000e+00 -3.55271368e-15  0.00000000e+00
  -3.55271368e-15 -3.55271368e-15  0.00000000e+00  0.00000000e+00
   0.00000000e+00 -3.55271368e-15]
 [-2.49800181e-16 -2.78943535e-15  1.77635684e-15 -3.55271368e-15
   3.55271368e-15 -3.55271368e-15  0.00000000e+00  0.00000000e+00
   0.00000000e+00  0.00000000e+00  3.55271368e-15 -3.55271368e-15
  -3.55271368e-15 -3.55271368e-15]
 [ 1.77635684e-15  1.77635684e-15 -3.55271368e-15  1.77635684e-15
   3.55271368e-15 -3.55271368e-15  0.00000000e+00  0.00000000e+00
   3.55271368e-15  7.10542736e-15  0.00000000e+00  3.55271368e-15
  -3.55271368e-15  3.55271368e-15]]
